# Introduction to Finite Difference Methods for Partial Differential Equations (PDEs)

Ordinary differential equations __(ODEs)__ have a single independent variable (and total derivatives).

 __Partial Differential Equations (PDEs)__ have more than one independent variable (and partial derivatives). 

Classification of __$2^{nd}$-order Quasi-linear PDEs__ that arise most frequently in engineering:

General form:
$$A(x,y) \frac{\partial^2 u}{\partial x^2}+2B(x,y) \frac{\partial^2 u}{\partial x \partial y}+C(x,y) \frac{\partial^2 u}{\partial y^2} = F(x,y,u, \frac{\partial u}{\partial x}, \frac{\partial u}{\partial y})$$

Typical independent variables: $x,y,z, \text{and } t$ 

Typical dependent variables: $u,v,w,\phi,\psi,\omega, \ldots$

Common shorthand: subscripts indicate partial derivatives:
$$A(x,y) \, u_{xx}+2B(x,y) \, u_{xy}+C(x,y) \, u_{yy} = F(x,y,u, u_x, u_y)$$

For __constant coefficients__, things simplify to:
$$A  u_{xx}+2B  u_{xy}+C  u_{yy} = F(x,y,u, u_x, u_y)$$

Usual classification:

- Focus on the highest ($2^{nd}$) order derivatives - - Consider __characteristic solutions__: $u = f(k x + y)$.

Chain rule $\implies u_x = k f', u_y = f'$

$f'(\cdot)$ means derivative w.r.t. argument

Plug into the PDE: $A k^2 f' +2 B k f' + c f' = 0$

Cancel $f' \implies$ quadratic equation to solve for k: 
$$A k^2 + 2 B k + C = 0$$

Major distinction: real or complex roots?

For real $k$:

- values of $u(k x + y)$ are constant along __real characteristic curves (or lines or directions)__ $k x + y =0 $

- information propagates along real characteristics

- "propagates" $\implies$ wave behavior

$\exist$ real characteristics if __discriminant__ $B^2 - A C \geq 0$

Classifications are summarized in the table below:



|   Type        |     Discriminant Value      |   Classic Example   |      Classic Equation    |    Normal Form Equation   |
| -------- | :------------------: | :--------: | :-------------: | :--------------: |
| Hyperbolic     |  $B^2-A C>0$  | Wave |  $u_{tt} = c^2 u_{xx}$     | $u_{vw} = f_1$ |
| Parabolic      |  $B^2-A C=0$  | Heat |  $u_{t} = c^2 u_{xx}$     | $u_{ww} = f_2$ |
| Elliptic      |  $B^2-A C<0$  | Laplace |  $u_{xx} + u_{yy} = 0$     | $u_{vv}+u_{ww} = f_3$ |

The different classes of equations have different properties $\implies$ choose appropriate numerical methods.

Hyperbolic PDEs have wave behavior: 

- Need to be careful about information propagation:
  - speed 
  - direction

So start with elliptic equations
- diffusion instead of wave propagation

## Finite Difference Discretization

Classic elliptic PDEs:
- Laplace equation with applications in:
  - electrostatics
  - fluid flow
  - gravitation
  - anything with a potential $\ldots$
- Becomes Poisson equation with source term

As with ODEs:
- floating-point arithmetic has no infinitesimals
- cannot actually compute derivatives
- so approximate derivatives using finite differences
- obtain finite difference equation

Start simply:
- 2D Cartesian domain (coords: $\{x,y\}$)
- Dirichlet BCs (values specified on boundary)
- Rectangular, coordinate-aligned domain

$$ \nabla^2 u = \frac{\partial^2u}{\partial x^2} + \frac{\partial^2u}{\partial y^2} = 0 \text{ on } x \in [0,L_x]; \; y \in [0,L_y]$$

BCs: $u(0,y)=u(L_x,y)=u(0,x)=0, \; u(L_y,x) = U$

As we did with ODEs:
- Discretize domain to produce a regular grid. 
  - Let $n_x$ be number of grid points along $x$ direction
    - $n_x -1$ intervals of length $\Delta x = L_x/(n_x -1)$
    - Specify a point by index `i`: $x_i = i \Delta x$
    - Compute values for $1\leq i\leq n_x -1$ corresponding to `i in range(1,nx-1)`
  - Similarly for $y$-direction:
    - $\Delta y = L_y/(n_y -1)$
    - $y_j = j \Delta y, \; 1 \leq j \leq n_y -1$

- Continuous domain $\to$ discrete grid of points: $(x_i, y_j)$
- Solve for values of dependent variable on the grid

$$u_{i,j} = u(x_i, y_j) = u\big( i \Delta x, j \Delta y \big)$$ 

- Now convert the PDE
  - Replace derivatives with central difference approximations
  $$
\begin{aligned}
\frac{\partial^2u}{\partial x^2} & \rightarrow \frac{1}{\Delta x^2}(u_{i-1,j}-2u_{i,j}+u_{i+1,j}) \\
\frac{\partial^2u}{\partial y^2} & \rightarrow \frac{1}{\Delta y^2} (u_{i,j-1} -2 u_{i,j} + u_{i,j+1})
\end{aligned}
$$
    - For each internal grid point get equation of the form:

$$ \nabla^2 u = \frac{\partial ^2 u}{\partial x^2} + \frac{\partial^2u}{\partial y^2} \rightarrow u_{i-1,j}+u_{i+1,j}+u_{i,j-1} + u_{i,j+1} -4u_{i,j} = 0$$

This quantity is more readable in code format:

`u[i-1,j] + u[i+1,j] + u[i,j-1] + u[i,j+1] - 4*u[i,j]`

- A 2D  __stencil computation__: 
  - Lay the 5-point stencil of coefficients below on top of a grid point
    - Coefficient $-4$ at the central grid point (red dot)
    - Coefficient $1$ at the neighboring grid points (black dots)
  - Compute "inner product" of stencil coefficients with the values at the covered grid points

> "5-point stencil" = __tensor product stencil of radius 1__: 1D stencil of radius 1 along each coordinate direction

![stencil_5point](stencil_5point.png)

With no source terms (so right-hand side is zero), we can solve for $u_{i,j}$ in terms of values at neighboring gridpoints:

$$u_{i,j} = \frac{1}{4} (u_{i-1,j} + u_{i+1,j} +u_{i,j-1} +u_{i,j+1} )$$

Two major choices from here:
- Assemble the equations for all of the interior grid points and solve one BIG system
  - Usually flatten array `u[i,j]` into 1D array
  - Eq. at each gridpoint is linear combination of flattened `u` array $\implies$ linear algebra problem
  - Can organize into banded matrix and employ special solver
- Treat equation at each grid point separately
  - Solve equation for value at that gridpoint
  $$u_{i,j} = \frac{1}{4} (u_{i-1,j} + u_{i+1,j} +u_{i,j-1} +u_{i,j+1} )$$
  - Numerical solution stisfies these conditions
  - Use as basis of iterative scheme indexed by superscript $k$
  - Use average of neighboring values at previous iteration as central value in next iteration:

$$u^{k+1}_{i,j} = \frac{1}{4} (u^k_{i-1,j} + u^k_{i+1,j} +u^k_{i,j-1} +u^k_{i,j+1} )$$

  - Repeatedly update each grid point. 
  - Converge to numerical solution!?
  - This approach is known as Jacobi iteration.

Implement this and see how it works: 
- HW3 Problem 4 (serial version)
  - doubly nested loop over grid
  - third loop over iteration count
- HW3 Problem 5 (basic parallel version)
  - launch 2D computational grid
  - single loop over iteration count

Why focus on finite difference methods when other methods (e.g. spectral methods based on Fourier analysis) can be more accurate?
  
### Non-axis-aligned BCs
- What if not all of the bounding rectangle is actually included in the domain?
- More generally, what if the boundary does not coincide with constant coordinate lines/curves?
- "Go to" method: finite element method (FEM)
  - Mesh the domain/discretize the boundary
  - Weak formulation involving integral over domain
  - Approximate solution as linear combination of trial/basis functions
  - Create large linear algebra problems to solve for coefficients
  - To date, parallel FEM just plugs in a parallel linear solver $\implies$ not so interesting from a parallel perspective
  - So back to finite difference methods


- Suppose domain is defined by implicit function 
$$f(x,y)<0$$ 
How to handle this? Several options:
  
- Evaluate $f$ at each grid point and apply update equation subject to  conditional:
```
if f(x[i],y[j]) < 0:
    #apply update formula
    u[i,j] = ...
```
- Alternatively update all grid points then reset conditions outside domain
  - Compute exterior values once and save
  - Re-compute exterior values repeatedly
- Consider implications of conditionals: 
  <br>`if` statements $\implies$ warp divergence

## Parabolic PDEs: <br>Stepping/marching methods

Move on to classic parabolic PDE: __Heat/diffusion equation__

$$ \frac{\partial u}{\partial t} = \kappa \frac{\partial^2 u}{\partial x^2} \text{ or } u_t = \kappa u_{xx}$$

- Describes evolution of temperature distribution in an insulated rod
- Independent variables: 
  - Spatial variable $x$
  - Time $t$

- Fundamental behavior indicated by free space Green's function (response to $\Delta$-function input on infinite domain):
  $$u(x,t) \sim \frac{1}{\sqrt{4 \pi k t}} e^{-x^2/(4 k t)} $$

  - Fixed amount of heat
  - Temperature decays $\to 0 \text{ as } x \to \pm \infty$
  - Temperature decays $\sim t^{-1/2} \text{ as } t \to \infty$

Again apply finite differences:

- Discretize both space and time: 
  $$x_i = i \Delta x \; ; \; t_k = k \Delta t$$

- Apply initial conditions at $t=0$: 
  $$u(x,0) = U_0(x)$$

- Estimate derivatives using finite differences.

Start with the simplest approach:

- Centered difference for spatial derivative:

$$\frac{\partial^2 u}{\partial x^2} \to \frac{1}{\Delta x^2}(u_{i-1,k}-2 u_{i,k} + u_{i+1,k})$$ 
- Forward difference for time derivative:
$$ \frac{\partial u}{\partial t} \to \frac{1}{\Delta t} (u_{i,k+1}-u_{i,k})$$

- Plug into PDE. 
- Note that only one term involves time $t_{k+1}$
- Solve for that term:

$$u_{i,k+1}= u_{i,k}+\frac{\Delta t}{\Delta x^2}(u_{i-1,k}-2 u_{i,k} + u_{i+1,k})$$

Translates into python as:

```
c = dt/dx**2
u[i,k+1] = u[i,k]+ c*(u[i-1,k]-2*u[i,k]+u[i,k+1])
```

- Explicit stepping/marching method: plug in values at time $t_k$, starting with $t=t_0=0$ where we have known initial values, and march forward to compute the next line of values $u(x_i, t_1)$
- Repeat to compute values for $u(x_i, t_k)$ over desired domain.

Again obtain a scheme where the iterations (here stepping forward in time instead of converging toward equilibrium) involve a stencil computation.

Note that this method is only stable for sufficiently small values of `c`.

Limited stepsize implies long computation times, so parallelizing can make a significant difference.

## Hyperbolic PDEs: Wave propagation

Start simpler with __one-way wave equation__

$$\frac{\partial u}{\partial t} = c_0 \frac{\partial u}{\partial x}$$

Recall that we could look for characteristic solutions of the form $u(x,t) = f(x+kt)$

$$k f' = c_0 f' \implies k = c_0$$

Solutions involve waves propagating with speed $c$ in which direction?

Again we start simple with the first central difference in $x$ and first forward difference in $t$:

`u[i,k+1] = u[i,k] + c*(u[i+1,k]-u[i-1,k])` 
<br>where `c = c0*dt/(2*dx)`.

Again we obtain explicit time-stepping scheme, but this time it turns out to be __Unstable for all stepsizes__

One fix involves ___implicit schemes___:

For implicit Euler in time with central difference in space, the discretized equation is:

$$u_j^{(m+1)} = u_j^{(m)} + c\big[u_{j+1}^{(m+1)}-u_{j-1}^{(m+1)}\big] \text{ where } c=\frac{c_0 \Delta t}{2 \Delta x}$$

This turns out to be stable, but at the cost of solving a linear system at every step.

Middle ground: Leapfrog method

$$u_j^{(m+1)} = u_j^{(m-1)} + c \big[u_{j+1}^{(m)}-u_{j-1}^{(m)}\big] $$

Explicit, but needs info from 2 previous time steps

Lots of details left to dicuss, but in all cases we can produce an updating/stepping scheme based on a stencil computation...


